<a href="https://colab.research.google.com/github/wangsouza/desafios/blob/main/Prova_02_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Utilizar visão computacional para identificar nos videos people-talking e people-walking pessoas falando e pessoas caminhando, 
gerando como saída um resultado similar aos vídeos people-talking-processed e people-walking-processed.

Obs: Explique o que está sendo realizado em cada etapa.



---



Para este desafio foi utilizado o classificador **Haar Cascade** para identificar rostos humanos e pessoas caminhando. O Haar cascade é uma abordagem eficaz de detecção de objetos proposta por Paul Viola e Michael Jones. Apesar de ter surgido em 2001, o seu desempenho tem evoluído com o passar dos anos. Atualmente, o Haar Cascade é bastante utilizados como base para sistemas mais avançados como reconhecimento facial, detector de fadiga, expressões faciais, e até mesmo detector de máscara facial.

1. Importa a biblioteca opencv

In [ ]:
import cv2

2. O desafio propõe a utilização de dois tipos de detectores (rostos e caminhada).
O trecho de código abaixo permite que o usuário selecione apenas a **opção 1 (detector de rosto)** e **2 (detector de caminhada)**.

In [ ]:
while True:
    try:
        opt = int(input("Digite:\n1. para acionar o detector de rostos.\n2. para acionar o detector de caminhada.\n"))
        if not 0 < opt <= 2:
            raise ValueError("Digite apenas os valores 1 ou 2.")
    except ValueError as e:
        print("Valor inválido:", e)
    else:
        break

Caso o usuário opte pelo **detector de rostos**, ele digitará a opção 1 e os arquivos necessários para próxima etapa serão carregados. O processo será o mesmo caso ele opte pela opção 2.
1. **CascadeClassifier** - Carrega o arquivo XML contendo os metadados de treinamento.
2. **VideoCapture** - carrega o arquivo de entrada presente no disco. Para usar a webcam, basta trocar por 0, ou 1, caso a porta esteja ocupada.
3. **fileNameOut** - nome do arquivo de saída (após o processamento).

In [ ]:
if opt == 1:
  file_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
  cap = cv2.VideoCapture('people-talking.mp4')
  fileNameOut = 'people-talking-processed.avi'
else:
  file_cascade = cv2.CascadeClassifier("haarcascade_fullbody.xml")
  cap = cv2.VideoCapture('people-walking.mp4')
  fileNameOut = 'people-walking-processed.avi'

Para salvar o vídeo após o processamento, é preciso especificar o código fourCC de 4 bytes necessário para compactar o vídeo. Neste desafio foi utilizado o codec MP4V. A lista completa está disponível em [fourcc.org](http://www.fourcc.org/codecs.php)
1. Resolução de saída
2. Selecionar o codec (compactação do vídeo)
3. Definir os parâmetros para salvar o vídeo como:
 * Nome do arquivo + extensão.
 * Codec.
 * Frames por segundo (FPS).
 * Resolução de saída.




In [ ]:
res=(640,360)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(fileNameOut, fourcc, 30.0, res)

Enquanto o vídeo estiver sendo executado, o laço While permanece como **True**.
1. **ret, img = cap.read()** - A primeira variável (**ret**) indica um valor booleano, quando o vídeo ainda estiver sendo executado o seu valor é True. A variável **img** corresponde ao fluxo de vídeo. A função read() realiza a leitura do arquivo de vídeo.


2. **if ret == False:break** - Ao chegar no fim da execução do vídeo a variável **ret** se torna False e a execução é imediatamente interrompida.

3. O próximo passo é carregar a imagem e converter de RBG para escala de cinza. A razão para isso é que o canal cinza garante menos custo computacional, pois contém apenas 1 canal, em vez de 3 canais RBGs.
     
     **cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)**
 * **img** - fluxo de vídeo
 * **cv2.COLOR_BGR2GRAY** - É o código de conversão do espaço de cores.

4. **file_cascade.detectMultiScale(gray, 1.1, 4)**
Neste trecho de código, utilizamos o file_cascade contendo os metadados de treinamento e instanciamos ao método detectMultiScale. Este método ajudará localizar possíveis características a cada frame da imagem.
Os parâmetros são:
 * **gray** - fluxo de vídeo em escala de cinza.
 * **scaleFactor** - parâmetro que especifica a escala que a imagem é reduzida. Ou seja, o modelo possui um tamanho definido durante o treinamento e está salvo no XML. Isso significa que se o objeto a ser detectado for maior que no arquivo XML, o fator de escala reduz o tamanho em 10%, baseado no valor 1.1, (5% no valor 1.05), e consequemente, aumentam as chances de coincidir com o modelo de detecção encontrado.
 * **minNeighbors** - especifica a quantidade de retângulos que serão retidos. Nessa fase, muitos falsos positivos poderão ocorrer, a função do minNeighbors é eliminar os falsos positivos. Valores maiores resultam em menos detecções.

5. A função **detecMultiScale** retorna 4 valores, a coordenada x, coordenada y, largura (w) e altura (h) do objeto. Com base nesses 4 valores, será desenhado o retângulo.

 **cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 255), 2)**
 Os parâmetros são:
 * **img** - fluxo de vídeo com 3 canais (RGB)
 * **(x, y)** - a localização/coordenada do retângulo.
 * **(x + w, y + h)** - as dimensões do retângulo.
 * **(0, 255, 255)** - a cor da borda do retângulo.
 * **2** -  a espessura da linha.

6. **out.write(img)** - a cada frame processado, as informações são gravadas no arquivo e compactadas com o codec previamente configurado.

7. **out.release()** e **cap.release()** - Fecha os arquivos de vídeo e os liberam para outros processos.



In [ ]:
while True:
  ret, img = cap.read()
  if ret == False:
    break
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  detections = file_cascade.detectMultiScale(gray, 1.1, 4)
  
  for (x, y, w, h) in detections:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 255), 2)
  out.write(img)
out.release()
cap.release()

É importante salientar que as configurações dos parâmetros scaleFactor, minNeighbors devem ser realizadas separadamente de forma empírica em cada projeto. Apesar de utilizarmos dois projetos no mesmo arquivo, o objetivo aqui foi economizar código. Pensando nisso, os parâmetros foram balanceados para que ambos tivessem os melhores desempenhos possíveis.
